In [77]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])


In [82]:
class SVCImpl(LinearClassifier):
    def __init__(self, n_iter=20, lr=0.01, regularizer=0.0):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lr = lr
        self.regularizer = regularizer

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                if y * score < 1:
                    self.w = (1 - self.regularizer * self.lr) * self.w + self.lr * y * x
                else:
                    self.w = (1 - self.regularizer * self.lr) * self.w

    def score(self, X, Y):
        score = 0.0
        for x, y in zip(X, self.encode_outputs(Y)):
            score -= float(np.max([0.0, 1.0 - y * x.dot(self.w)]))
        return score

In [83]:
class LogisticRegressionImpl(LinearClassifier):
    def __init__(self, n_iter=20, lr=0.01, probability=False, regularizer=0.0):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lr = lr
        self.probability = probability
        self.regularizer = regularizer

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

       # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):
                score = y * x.dot(self.w)
                self.w = (1 - self.regularizer * self.lr) * self.w + self.lr * y * self.sigmoid(-score) * x

    def score(self, X, Y):
        score = 0.0
        for x, y in zip(X, self.encode_outputs(Y)):
            score -= float(np.log(1 + np.exp(-y * x.dot(self.w))))
        return score

In [84]:

import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


# Read all the documents.
X, Y = read_data('pa4/data/all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    # NB that this is our LogisticRegression implementation
    SVCImpl()
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

pipeline.score(X, Y)

Training time: 2.63 sec.
Accuracy: 0.8363.


C:\Users\CYTech Student\.virtualenvs\Applied machine learning\lib\site-packages\numpy\core\fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


-4245.349436559636

In [85]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

param_grid = {
    'classifier__n_iter': [10, 30, 100, 300],
    'classifier__lr': [0.0001, 0.001, 0.01, 0.1],
    'classifier__regularizer': [0.01, 0.1, 1, 2]
}


pipeline = Pipeline(steps=[
    ("vect",TfidfVectorizer()),
    ("select", SelectKBest(k=1000)),
    ("norm", Normalizer()),
    ("classifier", SVCImpl())
])

# Grid search for hyperparameter C on the training data set
grid_search = GridSearchCV(pipeline, param_grid, n_jobs=-1, return_train_score=True)

grid_search.fit(X, Y)

print("Best parameter (loss=%0.3f):" % grid_search.best_score_)
print(grid_search.best_params_)

Best parameter (loss=-2315.679):
{'classifier__lr': 0.01, 'classifier__n_iter': 30, 'classifier__regularizer': 0.1}
{'mean_fit_time': array([2.19908748, 5.30850811]), 'std_fit_time': array([0.07327278, 1.02300263]), 'mean_score_time': array([0.81534724, 0.41111116]), 'std_score_time': array([0.04023157, 0.06440328]), 'param_classifier__lr': masked_array(data=[0.01, 0.01],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_classifier__n_iter': masked_array(data=[1, 30],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_classifier__regularizer': masked_array(data=[0.1, 0.1],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'classifier__lr': 0.01, 'classifier__n_iter': 1, 'classifier__regularizer': 0.1}, {'classifier__lr': 0.01, 'classifier__n_iter': 30, 'classifier__regularizer': 0.1}], 'split0_test_score': array([-2314.87732724, -2314.87240695]), 'split1_test_s